# Speech to Text

librerias requeridas

In [ ]:
import os
import io
from google.cloud import speech
# Imports the Google Cloud client library
from google.cloud import storage

llave de acceso a google, en caso de cambiar la cuenta de google se requiere crear nueva llave

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/covidpositivo-13fea1e69390.json"


variables globales 

In [ ]:
salida='/Path_donde_guarda_tmp_wav/out2.wav'

In [ ]:
idioma='es-CL'
pais_extraccion= 'Chile'
pattern = "*.mp4"
tipo_documento='Archivo mp4'

Para ajustar el formato del del audio desde mp4 a wav mono 16000

ffmpeg -i T1Mar1COVID.mp4 -ac 1 -ar 16000 out.wav

In [ ]:
def encode_audio(audio_entrada):
    path_base= "/"
    file=  audio_entrada #'T1Mar1COVID.mp4'
    comando = 'ffmpeg -i {} -ac 1 -ar 16000 {}'.format(file, salida)
    os.system(comando)

In [ ]:
#encode_audio('/home/andres/Alemania/RTL Aktuell 1/RTLAktuell_0510_COVID_02.mp4')

#ffmpeg -i nn.mp4 -ac 1 -ar 16000 misalidaError.wav

## Funciones para transcripción con google

In [ ]:
#funcion para transcripcion de video a texto
def transcribe_gcs(gcs_uri):
    """Asynchronously transcribes the audio file specified by the gcs_uri."""
    from google.cloud import speech

    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
       # encoding=speech.RecognitionConfig.AudioEncoding.FLAC,
        sample_rate_hertz=16000,
        language_code=idioma,
    )

    operation = client.long_running_recognize(
        request={"config": config, "audio": audio}
    )

    operation = client.long_running_recognize(config=config, audio=audio)

    print("Waiting for operation to complete...")
    response = operation.result(timeout=200)

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    datos=""
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        #print(u"Transcript: {}".format(result.alternatives[0].transcript))
        datos= datos + " " + result.alternatives[0].transcript
        #print("Confidence: {}".format(result.alternatives[0].confidence))
    return  datos


In [ ]:
#dato_final = transcribe_gcs("gs://nuevo_covid2/out2.wav")

In [ ]:
#print (dato_final)

In [ ]:
#storage_client = storage.Client()

In [ ]:
#funcion para subir archivo wav a bucket google
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )


In [ ]:
#upload_blob('nuevo_covid2', salida, 'out2.wav' )

In [ ]:
#print(hola)

In [ ]:
#elimina archivo desde bucket google
def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    # bucket_name = "your-bucket-name"
    # blob_name = "your-object-name"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.delete()

    print("Blob {} deleted.".format(blob_name))

In [ ]:
delete_blob('nuevo_covid', '../tmp_audio/out2.wav' )

## Procesamiento de archivos desde carpetas a base de datos

primero librerías requeridas para procesamiento de carpetas y bases de datos

In [ ]:
import ocrspace
import json
import requests
import os
from fnmatch import fnmatch
import pymssql

import time

Paso 1: extraer path de disco de los archivos a procesar

In [ ]:
archivos_procesar=[]
arreglo=[val for sublist in [[os.path.join(i[0], j) for j in i[2]] for i in os.walk('../chile/')] for val in sublist]
# Meta comment to ease selecting text
for name in arreglo:
        if fnmatch(name, pattern):
            archivos_procesar.append(name)


print(len(arreglo))
print(len(archivos_procesar))

In [ ]:
#Este código es sólo para verificar la estructura archivo con mayor cantodad de caracteres.
max=20
elmaslargo=''
for name in archivos_procesar:
    cantidad= len(archivos_procesar[2].split("/"))
    frase= archivos_procesar[2].split("/")
    if cantidad <max:
        max=cantidad
        elmaslargo=frase
print(max)
print(elmaslargo)

Acceso a la base de datos

In [ ]:
def conectar():
    conn = pymssql.connect("xxx.xxxx.xxx.xxx", "usuario_base", "clave_base", "nombre_base_datos")
    return conn

In [ ]:
tmp_arreglo= archivos_procesar

In [ ]:
for numero in range(len(tmp_arreglo)):
    print("pos " + str(numero) +"--" + tmp_arreglo[numero])

In [ ]:
print(len(tmp_arreglo))

archivos_procesar=tmp_arreglo[547:]
print(len(archivos_procesar))
print(archivos_procesar[0])
if not('\''  in archivos_procesar[0]):
    print("correcto")

In [ ]:
### PROCESAR PDFS #####

count=0
for fila2 in archivos_procesar:
        if not('\''  in fila2):
            print(count)
            print(fila2)
            encode_audio(fila2)
            time.sleep(20)
            print('listo tiempo espera')
            upload_blob('nuevo_covid2', salida, 'out2.wav' )
            dato_final = transcribe_gcs("gs://nuevo_covid2/out2.wav")
            delete_blob('nuevo_covid2', 'out2.wav' )
            pais= pais_extraccion
            frase_fila= fila2.split("/")
            fecha=frase_fila[2]  #'Sin fecha' #fila2.split("/")[-1].split(".")[0].split("_")[1]#fila.split("/")[-1].split(".")[0]
            medio= frase_fila[3]
            item1= 'otros'
            item2= 'otros'
            item3= 'otros'
            tipo=tipo_documento 
            nombre=fila2.split("/")[-1].split(".")[0]
            txt0=''
            txtc= dato_final
            txtt= 'pendiente'
            conn= conectar()
            cursor2 = conn.cursor()
            cursor2.executemany(
                            "INSERT INTO videos_covid VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                            [(pais,fecha,medio,item1,item2, item3,tipo,nombre,txt0,txtc,txtt)])
                            # you must call commit() to persist your data if you don't set autocommit to True
            conn.commit()
            cursor2.close()
            conn.close()
            comando = 'rm {}'.format(salida)
            os.system(comando)
            #print ('fin')
            #break
            count+=1



## Proceso Optativo

En algunos casos es necesario limpiar los nombres de archivos desde carácteres especiales, para ello los dós códigos a continuación lo permiten el primer código es para nombres de directorios y el útlimo código es para nombres de archivos. 

In [ ]:
import os
files = [x[0] for x in os.walk('../chile/')] 
cambio=' '
nuevo='_'
for src in files:
    if cambio in src:
        print(src)
        dst = src.replace(cambio, nuevo)
        os.rename(src, dst)

In [ ]:
print (files)

In [ ]:
archivos_procesar=[]
arreglo=[val for sublist in [[os.path.join(i[0], j) for j in i[2]] for i in os.walk('../chile/')] for val in sublist]
# Meta comment to ease selecting text
cambio=' '
nuevo='_'
for src in arreglo:
        if cambio in src:
            print(src)
            dst = src.replace(cambio, nuevo)
            os.rename(src, dst)